# ***Libraries & Tools***

In [ ]:
import sys
import numpy as np
import random
import networkx as nx
import tensorflow as tf
import scipy.cluster
import gc
import math
import shutil
import re
import os

from tensorflow.contrib import learn
from tensorflow.contrib import layers
from tensorflow.sparse import sparse_dense_matmul as sd_matmul

from datetime import datetime
from pathlib import Path
from tqdm import tqdm

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing

In [ ]:
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Optional: Set GPU memory growth to avoid over-allocation

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is ready for use!")
    except RuntimeError as e:
        print(e)


In [ ]:
print("Is TensorFlow using GPU?", tf.test.is_built_with_cuda())  # Should return True
print("GPU device:", tf.test.gpu_device_name())  # Should return something like /device:GPU:0

In [ ]:
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

# ***Variables & General Functionality***

In [ ]:
dataset_name    = "arxiv"
data_text_file  = "data-v3-500.txt" # For single execution
data_text_files = ["data-v3-500.txt", "data-v3-500C.txt", "YAKE10.txt", "YAKE5.txt", "RAKE10.txt", "RAKE5.txt", "RAKE10C.txt", "RAKE5C.txt", "TFIDF10.txt", "TFIDF5.txt", "PosR5.txt",
                   "PosR10.txt", "TextR5.txt", "TextR10.txt", "TopicR5.txt", "TopicR10.txt"]

graph_file             = 'graph.txt' # For node classification
parent_path            = f'Datasets/{dataset_name}/graph-v2'
log_file               = 'CANE_Execution_Logs.txt'
link_pred_results_file = 'CANE_Link_Pred_Res.txt'
node_clf_results_file  = 'CANE_Node_Clf_Res.txt'
categories_file        = 'group-v2.txt'
model_name = 'DetGP'


split_graph_file  = 'sgraph15.txt' # For single execution
split_graph_files = ['sgraph15.txt', 'sgraph45.txt', 'sgraph75.txt']

test_graph_file   = 'tgraph85.txt' # For single execution
test_graph_files  = ['tgraph85.txt', 'tgraph55.txt', 'tgraph25.txt']

In [ ]:
# Parameters for node classification
clf_ratio = [0.15, 0.45, 0.75]
clf_num = 5
train_classifier = True

In [ ]:
MAX_LEN = 300
MAX_LENS = []
neg_table_size = 1000000
NEG_SAMPLE_POWER = 0.75
batch_size = 64 # Orig: 128
num_epoch = 50
embed_size = 200
lr = 1e-3
inducing_num = 20
report_epoch_num = 1
eval_epoch_num = 10 * report_epoch_num
random_seed = 42

GPU_USAGE = 0.5
GPU_ID = 0

encoder_type = 'wavg'  # 'dmte' 'wavg'
kernel_type = 'linear'
trans_order = 3

In [ ]:
# Find the average number of words from each data text file
for txtf in data_text_files: # 1) ['data.txt'] 2) data_text_files:
    total_word_count = 0
    total_lines = 0

    with open(f'{parent_path}/{txtf}', 'r', encoding='utf-8') as file:
        for line in file:
            total_word_count += len(re.findall(r"\b\w+\b", line))
            total_lines += 1

    mean_word_count = total_word_count / total_lines if total_lines > 0 else 0
    MAX_LENS.append(int(math.ceil(mean_word_count)))
    print(f'=== {txtf} ===')
    print("Mean word count:", math.ceil(mean_word_count))
    print()


In [ ]:
MAX_LENS

In [ ]:
MAX_LEN = MAX_LENS[-1] # For single execution

In [ ]:
zero_list = []
for i in range(0, embed_size):
    zero_list.append(0)
zero_list = np.array(zero_list)

In [ ]:
def get_vectors_from_file(file_path):
  vectors = {}

  with open(f'{file_path}', "r") as f:
      for idx, line in enumerate(f):
          vector = list(map(float, line.strip().split()))  # Convert to list of floats
          vectors[idx] = vector  # Assign embedding to node idx

  return vectors

# ***Tools***

In [ ]:
def edges_to_undirect(node_num, edges):
    # When given edge [i,j] build undirect graph with edge [i,j] and [j,i], each edge only appear once
    # Create unique hash codes for the nodes in the edges
    edge_hash = []
    for edge in edges:
        edge_hash.append(edge[0]*node_num+edge[1])
        edge_hash.append(edge[1]*node_num+edge[0])
    
    edge_hash = list(set(edge_hash)) # edge_hash will have unique hash codes for the edges
    ud_edges = [[v // node_num, v % node_num] for v in edge_hash]
    return ud_edges

def get_initial_inducing(sess, model, inducing_num):
    text_feature = sess.run(model.text_feature)
    inducing_points = scipy.cluster.vq.kmeans2(text_feature, inducing_num, minit='points')[0]
    return inducing_points

# ***Negative Sample***

In [ ]:
def InitNegTable(edges):
    a_list, b_list = zip(*edges)
    a_list = list(a_list)
    b_list = list(b_list)
    node = a_list
    node.extend(b_list)

    node_degree = {}
    for i in node:
        if i in node_degree:
            node_degree[i] += 1
        else:
            node_degree[i] = 1
    sum_degree = 0
    for i in node_degree.values():
        sum_degree += pow(i, 0.75)

    por = 0
    cur_sum = 0
    vid = -1
    neg_table = []
    degree_list = list(node_degree.values())
    node_id = list(node_degree.keys())
    for i in range(neg_table_size):
        if ((i + 1) / float(neg_table_size)) > por:
            cur_sum += pow(degree_list[vid + 1], NEG_SAMPLE_POWER)
            por = cur_sum / sum_degree
            vid += 1
        neg_table.append(node_id[vid])

    return neg_table

# ***Dataloader***

In [ ]:
class DataLoader():
    def __init__(self, text_path, graph_path, edge_split_ratio=None):
        #self.graph, self.train_graph, self.test_graph = self.load_graph(graph_path, edge_split_ratio)
        text_file, graph_file = self.load(text_path, graph_path)
        self.edges = self.load_edges(graph_file)
        self.text, self.num_vocab, self.num_nodes = self.load_text(text_file)
        #self.train_edges = edges_to_undirect(self.num_nodes, self.train_graph.edges())
        #self.test_edges = edges_to_undirect(self.num_nodes, self.test_graph.edges())
        self.negative_table = InitNegTable(self.edges)
    
    def load(self, text_path, graph_path):
        text_file = open(text_path, 'rb').readlines()
        for a in range(0, len(text_file)):
            text_file[a] = str(text_file[a])
            
        graph_file = open(graph_path, 'rb').readlines()

        return text_file, graph_file

    def load_edges(self, graph_file):
        edges = []
        for i in graph_file:
            edges.append(list(map(int, i.strip().decode().split())))

        print("Total load %d edges." % len(edges))

        return edges

    def get_adj_list(self, edges, node_num):
        adj_list = []
        for node in range(node_num):
                nbr = [node]
                for edge in edges:
                        if node in edge:
                                nbr += edge
                adj_list.append(list(set(nbr)))
        return adj_list


    def load_graph(self, graph_path, edge_split_ratio):
        total_graph = nx.Graph()
        train_graph = nx.Graph()
        test_graph = nx.Graph()
        np.random.seed(random_seed)
        
        graph_file = open(graph_path, 'rb').readlines()
        for line in graph_file:
            edge = list(map(int, line.strip().decode().split()))
            total_graph.add_edge(edge[0],edge[1])
            
            if np.random.uniform(0.0, 1.0) <= edge_split_ratio:
                train_graph.add_edge(edge[0],edge[1])
            else:
                test_graph.add_edge(edge[0],edge[1])
                
        return total_graph, train_graph, test_graph

    # Original load_text() method
    """ def load_text(self, text_path):
        text_file = open(text_path, 'rb').readlines()
        vocab = learn.preprocessing.VocabularyProcessor(MAX_LEN)
        text = np.array(list(vocab.fit_transform(text_file)))
        num_vocab = len(vocab.vocabulary_)
        num_nodes = len(text_file)
        return text, num_vocab, num_nodes """
    
    def load_text(self, text_file):
        #text_file = open(text_path, 'rb').readlines()
        #for a in range(0, len(text_file)): text_file[a] = str(text_file[a])

        vectorize_layer = tf.keras.layers.TextVectorization(
            max_tokens=None,  # Set a limit if needed
            output_mode='int',
            output_sequence_length=MAX_LEN
        )

        text_data = [line.strip() for line in text_file]
        vectorize_layer.adapt(text_data)
        text = vectorize_layer(text_data).numpy()
        return text, len(vectorize_layer.get_vocabulary()), len(text)

    
    def subgraph_edges(self, node_list):
        subg_edges = self.train_graph.subgraph(node_list).edges()
        return edges_to_undirect(self.num_nodes, subg_edges)

    # Original negative_sampling() method
    """ def negative_sampling(self, graph, edges):
        node1, node2 = zip(*edges)
        sample_edges = []
        #np.random.seed(config.random_seed)
        for i in range(len(edges)):
            neg_node = random.choice(list(graph.nodes))
            while neg_node in graph.neighbors(node1[i]):  
                neg_node = random.choice(list(graph.nodes))
            sample_edges.append([node1[i], node2[i], neg_node])
        return sample_edges """
    
    def negative_sampling(self, edges):
        node1, node2 = zip(*edges)
        sample_edges = []
        func = lambda: self.negative_table[random.randint(0, neg_table_size - 1)]
        for i in range(len(edges)):
            neg_node = func()
            while node1[i] == neg_node or node2[i] == neg_node:
                neg_node = func()
            sample_edges.append([node1[i], node2[i], neg_node])

        return sample_edges


    def generate_batches(self, mode=None):
        num_batch = len(self.train_edges) // batch_size
        edges = self.train_edges

        if mode == 'add':
          num_batch += 1
          edges.extend(edges[:(batch_size - len(self.edges) // batch_size)])

        if mode != 'add':
          random.shuffle(edges)

        sample_edges = edges[:num_batch*batch_size]
        sample_edges = self.negative_sampling(self.train_graph, sample_edges)

        batches = [sample_edges[i * batch_size:(i + 1) * batch_size] for i in range(num_batch)]
        return batches


# ***Model***

In [ ]:
def RBF_Kernel(feature_a, feature_b, gamma = 1000.):
    f_a = tf.expand_dims(feature_a, 1)  #[node_num_a, 1, f_dim]
    f_b = tf.expand_dims(feature_b, 0)  #[1, node_num_b, f_dim]
    k_ab = tf.reduce_sum(tf.square(f_a-f_b), axis = 2)
    return tf.exp(-k_ab / gamma)

def Linear_Kernel(feature_a, feature_b):
    k_ab = tf.matmul(feature_a, feature_b, transpose_b = True)
    return k_ab + 1.


def Polynomial_Kernel(feature_a, feature_b):
    return Linear_Kernel(feature_a, feature_b)**2.


def regularized_adj(adj):
    return adj/tf.sparse.reduce_sum(adj, axis = 1, keepdims = True)


def get_transition_matrix(Adj): # a sparse adjacency matrix
    reg_Adj = tf.sparse.add(Adj, tf.sparse.eye(tf.shape(Adj)[0]))
    return reg_Adj/tf.sparse.reduce_sum(reg_Adj,axis = 1, keepdims = True)


def norm_trans(adj_mat):
    return adj_mat / tf.sparse.reduce_sum(adj_mat, axis = 1, keepdims = True)
    

class DetGP():
    def __init__(self, vocab_size, num_nodes, text_data, edges):
        self.num_nodes = num_nodes
        self.kernel = Linear_Kernel
        self.inducing_num = inducing_num
        self.whiten = False
        self.embedding_dim = embed_size // 2
        self.trans_order = trans_order

        with tf.name_scope('read_inputs') as scope:
            self.text_all = tf.constant(text_data, dtype = tf.int32)
            self.node_a_ids = tf.compat.v1.placeholder(tf.int32, [None], name = 'a_ids')
            self.node_b_ids = tf.compat.v1.placeholder(tf.int32, [None], name = 'b_ids')
            self.node_n_ids = tf.compat.v1.placeholder(tf.int32, [None], name = 'n_ids')

            self.edges = tf.constant(edges, dtype=tf.int64)
            self.adj_mat = tf.sparse.SparseTensor(
                            self.edges, tf.ones(tf.shape(self.edges)[0]),
                            dense_shape=[self.num_nodes, self.num_nodes])
            self.trans_mat = get_transition_matrix(self.adj_mat)
            #self.initial_inducing_points = tf.placeholder(tf.float32, [None, embed_size/2], name = 'inducing')

        with tf.variable_scope('ggp') as scope:
            self.inducing_points = tf.get_variable(name = 'inducing_points', shape = [self.inducing_num, self.embedding_dim])
            self.q_mu            = tf.get_variable(name = 'embedding_mu', shape = [self.inducing_num, self.embedding_dim], initializer = tf.initializers.constant(0.1))
            self.alpha           = tf.get_variable(name = 'alpha', shape = [trans_order + 1], initializer = tf.initializers.constant(0.))
            self.al              = tf.nn.softmax(self.alpha)

        with tf.name_scope('initialize_embedding') as scope:
            self.text_embed =   tf.Variable(tf.truncated_normal([vocab_size, embed_size // 2], stddev=0.3), name = 'word_embedding')
            # self.node_embed = tf.Variable(tf.truncated_normal([num_nodes, embed_size // 2], stddev=0.3))
            # self.node_embed = tf.clip_by_norm(self.node_embed, clip_norm=1, axes=1)

        with tf.name_scope('lookup_embeddings') as scope:
            self.text_emb_lookup = tf.nn.embedding_lookup(self.text_embed, self.text_all)
            self.text_feature = tf.reduce_mean(self.text_emb_lookup, axis=1)               

        self._build_model()


    def load_inducing_points(self, inducing_points):
        self.inducing_points = tf.assign(self.inducing_points, inducing_points)
        
        
    def ggp_proces (self, features, z_features):
        delta = 0.01
        Kxz = self.kernel(features, z_features)
        Kzz = self.kernel(z_features, z_features) +tf.eye(self.inducing_num) * delta
        Kzz = tf.stop_gradient(Kzz)

        if self.trans_order == 3:
            Kxz_0 = Kxz
            Kxz_1 = sd_matmul(self.trans_mat, Kxz)
            Kxz_2 = sd_matmul(self.trans_mat, sd_matmul(self.trans_mat, Kxz))
            Kxz_3 = sd_matmul(self.trans_mat, Kxz_2)
            Kxz = self.al[0]*Kxz_0 + self.al[1]*Kxz_1 + self.al[2] * Kxz_2 + self.al[3]*Kxz_3
            
        Lz = tf.cholesky(Kzz)
        
        # Compute the projection matrix A
        A = tf.matrix_triangular_solve(Lz, tf.transpose(Kxz), lower=True)
        if not self.whiten:
            A = tf.matrix_triangular_solve(tf.transpose(Lz), A, lower=False)
        
        # construct the conditional mean
        fmean = tf.matmul(A, self.q_mu, transpose_a=True)        
        return fmean


    def compute_loss(self, emb_a, emb_b, emb_n):
        positive_loss = tf.reduce_sum(emb_a * emb_b, axis = 1)
        negative_loss = -tf.reduce_sum(emb_a * emb_n, axis = 1)
        p_likeli = positive_loss - tf.math.softplus(positive_loss)     #log(sigmoid(x))
        n_likeli = negative_loss - tf.math.softplus(negative_loss)

        total_loss = -tf.reduce_mean(p_likeli + n_likeli)
        return total_loss


    def get_distance(self, feature, ind_points):
        feature_prime = tf.expand_dims(feature, axis = 1)
        ind_points_prime = tf.expand_dims(ind_points, axis = 0)
        dist = tf.reduce_mean(tf.square(feature_prime - ind_points_prime), axis = 2) #[feature_num, induc_num]
        return tf.reduce_max(tf.sqrt(dist))

    
    def dmte_embeddings(self, text_features):
        first_order_emb = text_features
        second_orde_emb = sd_matmul(self.trans_mat, self.text_feature)

        return first_order_emb + second_orde_emb

    
    def _build_model(self):
        self.text_emb = self.dmte_embeddings(self.text_feature)
        self.struct_emb = self.ggp_proces(self.text_emb, self.inducing_points)

        self.text_emb_a = tf.nn.embedding_lookup(self.text_emb, self.node_a_ids)
        self.text_emb_b = tf.nn.embedding_lookup(self.text_emb, self.node_b_ids)
        self.text_emb_n = tf.nn.embedding_lookup(self.text_emb, self.node_n_ids)
        self.text_loss  = self.compute_loss(self.text_emb_a, self.text_emb_b, self.text_emb_n)

        self.struct_emb_a = tf.nn.embedding_lookup(self.struct_emb, self.node_a_ids)
        self.struct_emb_b = tf.nn.embedding_lookup(self.struct_emb, self.node_b_ids)
        self.struct_emb_n = tf.nn.embedding_lookup(self.struct_emb, self.node_n_ids)
        self.struct_loss  = self.compute_loss(self.struct_emb_a, self.struct_emb_b, self.struct_emb_n)
        
        self.total_loss = self.text_loss + 0.3 * self.struct_loss



# ***Classify***

In [ ]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            probs_[:] = 0
            probs_[labels] = 1
            all_labels.append(probs_)
        return np.asarray(all_labels)


class Classifier(object):

    def __init__(self, vectors, clf):
        self.embeddings = vectors
        self.clf = TopKRanker(clf)
        self.binarizer = MultiLabelBinarizer(sparse_output=True)

    def train(self, X, Y, Y_all):
        self.binarizer.fit(Y_all)
        # X_train = [self.embeddings[x] for x in X]
        X_train = [self.embeddings[int(x)] for x in X] # For each node in X, take its embedding
        Y = self.binarizer.transform(Y)
        self.clf.fit(X_train, Y)

    def evaluate(self, X, Y):
        top_k_list = [len(l) for l in Y] # For each label in Y, take its size (multi-label)
        Y_ = self.predict(X, top_k_list)
        Y = self.binarizer.transform(Y)
        averages = ["micro", "macro"]
        results = {}
        for average in averages:
            results[average] = f1_score(Y, Y_, average=average)
        return results

    def predict(self, X, top_k_list):
        X_ = np.asarray([self.embeddings[int(x)] for x in X])
        Y = self.clf.predict(X_, top_k_list=top_k_list)
        return Y

    def split_train_evaluate(self, X, Y, train_precent, seed=0):
        state = np.random.get_state()
        training_size = int(train_precent * len(X)) # Set the ratio based on the size of X
        np.random.seed(seed)
        shuffle_indices = np.random.permutation(np.arange(len(X))) # Shuffle the indices of X (X contains all nodes)

        # Access the values of X and Y based on the shuffled indices

        # X_train and Y_train will have "training_size" number of values of X and Y
        X_train = [X[shuffle_indices[i]] for i in range(training_size)]
        Y_train = [Y[shuffle_indices[i]] for i in range(training_size)]

        # X_test and Y_test will have "len(X) - training_size" number of values of X and Y
        X_test = [X[shuffle_indices[i]] for i in range(training_size, len(X))]
        Y_test = [Y[shuffle_indices[i]] for i in range(training_size, len(X))]

        self.train(X_train, Y_train, Y) # Y has the labels of all nodes
        np.random.set_state(state)
        return self.evaluate(X_test, Y_test)

# ***Run (Single Execution)***

In [ ]:
label_dic = {}
with open(f'{parent_path}/{categories_file}', 'r') as f:
  labels = f.readlines()

for la in labels:
  label_dic[la.split()[0]] = la.split()[1:][0] # la.split()[0] = the node id ----- la.split()[1:][0] = The label of that node. If a node has many labels, take the first


In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_USAGE)

In [ ]:
data = DataLoader(f'{parent_path}/{data_text_file}', f'{parent_path}/{graph_file}')

In [ ]:
# Saving embeddings
#embed_file = f"{parent_path}/Results/DetGP/embed_link_pred_{split_graph_file.split('.')[0]}_{data_text_file.split('.')[0]}.txt"
embed_file = f"{parent_path}/Results/DetGP/embed_node_clf_{graph_file.split('.')[0]}_{data_text_file.split('.')[0]}.txt" # For node classification the whole graph ('graph.txt') is used

In [ ]:
with tf.Graph().as_default():
    sess = tf.compat.v1.Session(config=tf.ConfigProto(gpu_options=gpu_options))

    with sess.as_default():
        model = DetGP(data.num_vocab, data.num_nodes, data.text, data.edges)
        opt = tf.compat.v1.train.AdamOptimizer(lr)
        train_op = opt.minimize(model.total_loss)
        sess.run(tf.compat.v1.global_variables_initializer())

        inducing_points = get_initial_inducing(sess, model, inducing_num)
        model.load_inducing_points(inducing_points)

        # Training
        start_time = datetime.now()
        for epoch in range(num_epoch):
            loss_epoch=0
            batches=data.generate_batches()
            num_batch=len(batches)
            for i in range(num_batch):
                batch=batches[i]

                node1, node2, node3=zip(*batch)    
                node1, node2, node3=np.array(node1),np.array(node2),np.array(node3)
                # text1, text2, text3 = data.text[node1], data.text[node2], data.text[node3]
                feed_dict={
                    #model.edges: data.edges,
                    #model.text_all: data.text,
                    #model.inducing_points: inducing_points,
                    model.node_a_ids: node1,
                    model.node_b_ids: node2,
                    model.node_n_ids: node3}

                # run the graph 
                _, loss_batch, al  = sess.run([train_op, model.total_loss, model.al],feed_dict=feed_dict)
                loss_epoch += loss_batch
        
        end_time = datetime.now()
        print(f'Total time: {((end_time - start_time).total_seconds()) / 60.0} min')

        text_emb, struct_emb = sess.run([model.text_emb_a, model.struct_emb_a], feed_dict = {
            # model.edges: data.edges,
            # model.text_all: data.text,
            model.node_a_ids: np.arange(data.num_nodes) 
        })
        
        embed = np.concatenate((text_emb, struct_emb), axis=1)

        with open(embed_file, 'wb') as f:
            for i in range(data.num_nodes):
                if embed[i]:
                    f.write((' '.join(map(str, embed[i])) + '\n').encode())
                else:
                    #f.write('\n'.encode()) # For link prediction
                    f.write((' '.join(map(str, zero_list)) + '\n').encode()) # For node classification

gc.collect()